In [ ]:
import os
import re
import openai
from utils import write_text_to_file, number_to_four_digit_string
from time import sleep
from config import name_of_book, full_book_txt, client, model_to_use, backup_model

count = 1


In [ ]:
chunk_directory=os.path.join(name_of_book,"text_chunks")

with open(full_book_txt, 'r', encoding='utf-8') as file:
    remaining_text = full_text = file.read()

if os.path.exists(chunk_directory):
    files = [f for f in os.listdir(chunk_directory) if os.path.isfile(os.path.join(chunk_directory, f))]
    # Sort files alphabetically
    files.sort()
    # Return the last file if there are any files
    if files:
        filepath = os.path.join(chunk_directory, files[-1])

        name_of_file = files[-1]
        without_the_other_stuff = int(name_of_file.replace(name_of_book,"").replace(".txt",""))
        count += without_the_other_stuff

        with open(filepath, 'r', encoding='utf-8') as file:
            finished_text = file.read()

            length_to_cut = remaining_text.find(finished_text)
            length_to_cut += len(finished_text)

            remaining_text=remaining_text[length_to_cut:]

# print(remaining_text)
percent_complete = 100 - (len(remaining_text)/len(full_text))*100
percent_complete = round(percent_complete, 2)
print(percent_complete)

In [ ]:
assistant = client.beta.assistants.create(
    name="Literature Chunker",
    instructions="You are meant to divide text into segments based on natural breaks in the narrative, ensuring chunks can stand alone with its accompanying illustration",
    tools=[],
    model=model_to_use,
)

assistant2 = client.beta.assistants.create(
    name="Literature Chunker",
    instructions="You are meant to divide text into segments based on natural breaks in the narrative, ensuring chunks can stand alone with its accompanying illustration",
    tools=[],
    model=backup_model,
)

def limit_context_window(upgrade=False):

    thread = client.beta.threads.create()

    prompt = "You are meant to divide text into segments based on natural breaks in the narrative, ensuring chunks can stand alone with its accompanying illustration. Do not give a break that causes a sentence to be split in half. A few examples are: "
    prompt += '''\n"No, it mustn't be so any more; we must not part like that! what are two evenings?"'''
    prompt += '\n"I bless my luck for the excellent knotted stick, which happened on that occasion to be in my right hand."'
    prompt += '\n"And timid as I was with women, yet this was such a moment!"'

    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt
    )

    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id if upgrade == False else assistant.id,
        instructions="Please follow the user's instructions and acknowlege that you understand.",
        # event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
        limit=1

    )

    return thread

In [ ]:
def get_length_to_cut(literature, count=0):
    prompt = "Okay, find the first natural break in this snippet, where you think it would be appropriate to add an accompanying illustration."
    prompt += "The intention is that the chunks can stand alone with its accompanying illustration."
    prompt += "Tell me where the break is by telling me the last ten or so words from the snippet that marks the break."
    prompt += "**Enclose the last sentence** between two triple hashes, like ###At this point Nastenka stopped and began laughing.###"
    prompt += "**Make sure that you do not cut off a word or a sentence**."
    prompt += "Do not capitalize anything unless it was already capitalized. Do not lowercase anything unless it was already lowercased."
    prompt += "Do not give me multiple breaks."
    prompt += "Take all the time you need to get it done right the first time. It's better to be late than incorrect."
    prompt += "I encourage you to think aloud to decide where the right break is."
    prompt += "Please, this is for spreading literature to the masses."

    # if count > 20: prompt += f"This is not the first time I have asked variants of you for it, and it's still not working. Please look for the {round(count/10) + 1}th best solution"

    thread = limit_context_window(upgrade=False if count < 10 else True)

    message = client.beta.threads.messages.create(
        thread_id=thread.id,
        role="user",
        content=prompt+"'''" + literature + "'''"
    )

    with client.beta.threads.runs.stream(
        thread_id=thread.id,
        assistant_id=assistant.id,
        instructions="Please follow the user's instructions.",
        # event_handler=EventHandler(),
    ) as stream:
        stream.until_done()

    messages = client.beta.threads.messages.list(
        thread_id=thread.id,
        limit=1

    )
    # type(messages)
    length_to_cut = 0
    for message in messages:
        raw_response = message.content[0].text.value

        pattern = r'###(.*?)###'

        # Use re.search to grab the first occurrence
        match = re.search(pattern, raw_response)

        if not match:
            if count % 40 == 0 and count != 0: print("\n")
            print("🙅",end="")
            return get_length_to_cut(literature, count=count+1)
        text_that_got_returned = match.group(1)

        if len(text_that_got_returned) < 30:
            if count % 40 == 0 and count != 0: print("\n")
            print("🤏",end="") 
            return get_length_to_cut(literature, count=count+1)
        if text_that_got_returned[-1] not in [".","!","?",'"']:
            if count % 40 == 0 and count != 0: print("\n")
            print("🔚",end="")
            return get_length_to_cut(literature, count=count+1)
        if text_that_got_returned[-1] == text_that_got_returned[-2] == ".":
            if count % 40 == 0 and count != 0: print("\n")
            print("😶",end="")
            return get_length_to_cut(literature, count=count+1)


        # print(text_that_got_returned)
        length_to_cut = literature.find(text_that_got_returned)
        if length_to_cut == -1:
            if count % 40 == 0 and count != 0: print("\n")        
            print("😱",end="")
            return get_length_to_cut(literature, count=count+1)

        
        print(text_that_got_returned)
        length_to_cut += len(text_that_got_returned)
        break    

    return length_to_cut

In [ ]:
chunks = []
window_size = 1500
while len(remaining_text) > 0:

    literature = remaining_text[:window_size]
    remaining_text = remaining_text[window_size:]
    
    try:
        length_to_cut = get_length_to_cut(literature)
    except openai.APIError as e:
        sleep(30)
        length_to_cut = get_length_to_cut(literature, count=0)
    
    chunk = literature[:length_to_cut]
    remainder = literature[length_to_cut:]
    chunks.append(chunk)

    remaining_text = remainder + remaining_text

    write_text_to_file(chunk_directory, number_to_four_digit_string(count), literature[:length_to_cut])
    count+=1
    # we're going to print the percent complete, to two decimal places
    percent_complete = 100 - (len(remaining_text)/len(full_text))*100
    percent_complete = round(percent_complete, 2)
    print(f"### Percent Complete: {percent_complete}%")

    